### Connect to gdrive

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/MyDrive

### Install mimikit

In [ ]:
!pip install git+https://github.com/ktonal/mimikit@experiment/new-data-again

### cd into project's repo

In [ ]:
%cd AIMC-Submission-2021

## Imports and setup neptune

In [ ]:
import matplotlib.pyplot as plt
import torch 

from mimikit.models.freqnet import FreqNet
from mimikit import get_trainer, audio, NeptuneConnector
from mimikit.audios import transforms as A

nc = NeptuneConnector(user="k-tonal",
                      setup={
                          "model": "experiment-Stimme",
                          "trained": "experiment-Stimme/EXS-1"
                      })

## Make db from list of files

In [ ]:
from dbs import DBS

# Add your files combinations in dbs.py and load them here

DBS

In [ ]:
##########  Options for creating DBS ###########

N_FFT = 256
HOP_LENGTH = 64

# for instance :

files = DBS["yodel"]

# or when you downloaded a model :

# files = net.hparams.files

# or directly so :

# files = ["Laura Newton.m4a", "Perotin.mp3", "Stimmung.mp3"]

###################################################

files_paths = ["./data/" + file for file in files]

db = FreqNet.db_class.make("/content/tmp-db.h5", files=files_paths, sr=22050,
                           n_fft=N_FFT, hop_length=HOP_LENGTH)

db

### download and load model

In [ ]:
# nc.download_experiment("trained", destination="/content/", artifacts="states/")

# net = FreqNet.load_from_checkpoint("/content/" + nc.setup["trained"].split("/")[-1] + "/states/epoch=2.ckpt", db=db)

# net

In [ ]:
net = FreqNet(db=db,
              
              n_layers=(8,),
              kernel_size=2,
              
              gate_dim=512,  # == model_dim
              skip_dim=None,  # None == without skips, with skips == int (256, 512...)
              residuals_dim=None, # None == without residuals, with reisduals == int (256, 512...)
              groups=1,
              accum_outputs=0,
              pad_input=0,
              
              batch_seq_length=512,  # must be > als receptive_field
              
              max_lr=4e-4,
              betas=(.9, .9),
              div_factor=1.,
              final_div_factor=1000.,
              pct_start=0.,
              batch_size=64,
              input_dim=db.fft.shape[1],
               )

net.hparams.files = files
net.hparams.model_class = "FreqNet"

net.receptive_field, net.hparams

### trainer

In [ ]:
trainer = get_trainer(root_dir="/content/freqnet",
                      max_epochs=100,
                      epochs=[99],
                     # comment these if you don't want to track with neptune :
                     model=net,
                     neptune_connector=nc,
                     )

## train

In [ ]:
trainer.fit(net)

# nc.upload_model("model", net, artifacts=("states", ))

In [ ]:
########### PLAY WITH THOSE : ###################

dl = iter(net.datamodule.train_dataloader())
inpt, trgt = next(dl)
new = inpt[0:8].clone().to("cuda")


n_steps = 2000

###############################################

## LOS GEHT'S!

new = net.generate(new, n_steps, decode_outputs=True)


for i in range(new.size(0)):

    y = new[i].squeeze().cpu().detach().numpy()
    print("prompt number", i)
    plt.figure(figsize=(20, 4))
    plt.plot(y)
    plt.show()

    audio(y, sr=22050)

### log selected prompts to neptune

In [ ]:
# ...
from random import randint

numbers_to_log = [0, 3, 5]

for i in numbers_to_log:
    y = new[i].squeeze().cpu().numpy()
    
    net.log_audio("output_id%i" % randint(0, 1e5), y, sample_rate=16000)